In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
%load_ext autotime

time: 0 ns (started: 2023-06-30 15:54:36 +07:00)


In [3]:
import torch
torch_version = torch.__version__
assert float(torch_version[:3]) >= 1.5
torch_version

'2.0.0+cu117'

time: 0 ns (started: 2023-06-30 15:54:38 +07:00)


In [4]:
from pathlib import Path as p; p.cwd()  # make sure we know where we are

WindowsPath('d:/NC_LAB/Trash_Classification/Code/DETR')

time: 16 ms (started: 2023-06-30 15:54:40 +07:00)


In [5]:
%ls

 Volume in drive D is New Volume
 Volume Serial Number is 02E1-90BB

 Directory of d:\NC_LAB\Trash_Classification\Code\DETR

06/29/2023  10:50 AM    <DIR>          .
06/28/2023  10:05 AM    <DIR>          ..
06/29/2023  09:44 AM    <DIR>          detr
06/28/2023  04:27 PM    <DIR>          DETR_Output
06/30/2023  03:54 PM            20,874 DETR_Training.ipynb
06/29/2023  10:50 AM       166,595,797 detr-r50_training.pth
06/28/2023  04:34 PM       166,618,694 detr-r50-e632da11.pth
06/29/2023  10:50 AM    <DIR>          logdirs
01/29/2022  09:10 AM    <DIR>          splits_final_deblurred
06/28/2023  10:07 AM    <DIR>          training-detr
06/28/2023  10:08 AM    <DIR>          zerowaste
               3 File(s)    333,235,365 bytes
               8 Dir(s)  74,612,064,256 bytes free
time: 15 ms (started: 2023-06-30 15:54:47 +07:00)


In [6]:
from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

from torch import nn
import torchvision.transforms as T

time: 703 ms (started: 2023-06-30 15:54:50 +07:00)


In [7]:
# Loading the weights, clipping out the two layers we want to train and saving it
detr_model = torch.load("detr-r50-e632da11.pth", map_location='cpu')
type(detr_model)

dict

time: 125 ms (started: 2023-06-30 15:54:53 +07:00)


In [ ]:
from torchinfo import summary
batch_size = 1
summary(detr_model, input=(batch_size, 3, 600, 600))

In [ ]:
new_base = list(detr_model.children())
print(new_base)

In [ ]:
import torch
from torch.nn import Linear
new_base[1] = Linear(in_features=256, out_features = 5, bias=True)
print(new_base[1])

In [ ]:
print(new_base)

In [ ]:
detr_state = torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth",
                                                map_location="cpu")
type(detr_state)

In [8]:
del detr_model['model']['class_embed.weight']
del detr_model['model']['class_embed.bias']
torch.save(detr_model, 'detr-r50_training.pth')

time: 188 ms (started: 2023-06-30 15:55:52 +07:00)


In [10]:
%cd detr

d:\NC_LAB\Trash_Classification\Code\DETR\detr
time: 0 ns (started: 2023-06-30 15:57:55 +07:00)


In [13]:
!python -m main.py --dataset_file "mycoco" --coco_path "D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/" --output_dir "D:/NC_LAB/Trash_Classification/Code/DETR/DETR_Output" --resume "D:/NC_LAB/Trash_Classification/Code/DETR/detr-r50_training.pth" --lr 1e-5 --lr_backbone 1e-6 --epochs 1

time: 2.64 s (started: 2023-06-30 15:59:11 +07:00)


d:\anaconda3\python.exe: Error while finding module specification for 'main.py' (ModuleNotFoundError: __path__ attribute not found on 'main' while trying to find 'main.py'). Try using 'main' instead of 'main.py' as the module name.


In [9]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path

import numpy as np
from torch.utils.data import DataLoader, DistributedSampler

import detr
from detr.datasets import build_dataset, get_coco_api_from_dataset
import detr.util.misc as utils
from detr.engine import evaluate, train_one_epoch
from detr.models import build_model

ModuleNotFoundError: No module named 'datasets'

time: 343 ms (started: 2023-06-30 15:56:27 +07:00)


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, cv2, random
import io

# import torch and torchvision
from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, MapDataset
from detectron2.data.datasets import load_coco_json, register_coco_instances
from pathlib import Path
from detectron2.checkpoint import DetectionCheckpointer

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
dataset_train_name = "zero-waste-train"
dataset_val_name = "zero-waste-val"
dataset_test_name = "zero-waste-test"


from detectron2.data.datasets import register_coco_instances
register_coco_instances(dataset_train_name, {}, "D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/train/labels.json", "D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/train/data")
register_coco_instances(dataset_val_name, {}, "D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/val/labels.json", "D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/val/data")
register_coco_instances(dataset_test_name, {}, "D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/test/labels.json", "D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/test/data")

In [ ]:
# Train data
waste_train_dict = load_coco_json("D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/train/labels.json", "D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/train/data",
                dataset_name=dataset_train_name)

# Val data
waste_val_dict = load_coco_json("D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/val/labels.json", "D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/val/data",
                dataset_name=dataset_val_name)

# Test data
waste_test_dict = load_coco_json("D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/test/labels.json", "D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/test/data",
                dataset_name=dataset_test_name)

train_metadata = MetadataCatalog.get(dataset_train_name)
test_metadata = MetadataCatalog.get(dataset_test_name)
val_metadata = MetadataCatalog.get(dataset_val_name)

In [ ]:
from detectron2.data.transforms import ScaleTransform

# Resize image in train_dict
scale_transform = ScaleTransform(1080, 1920, 540, 960, "nearest")

# Define the dataset mapper to apply the Scale transform
def dataset_mapper(dataset_dict):
    for d in dataset_dict:
        # Apply the Scale transform to the image
        image = d["file_name"]
        image = scale_transform.apply_image(image)

        # Create a new dataset dictionary with the scaled image
        d["file_name"] = image

    # Return the modified dataset dictionary
    return dataset_dict

In [ ]:
dataset_train = MapDataset(DatasetCatalog.get(dataset_train_name), dataset_mapper)
dataset_val = MapDataset(DatasetCatalog.get(dataset_val_name), dataset_mapper)
dataset_test = MapDataset(DatasetCatalog.get(dataset_test_name), dataset_mapper)

In [ ]:
for d in random.sample(waste_train_dict, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    print(d["file_name"])
    print(f"Actual image shape: {img.shape}")
    vis_image = out.get_image()

    # Get the shape of the image
    image_shape = vis_image.shape

    print(f"Scaled image shape: {image_shape}")
    plt.imshow(out.get_image()[:, :, ::-1])

In [ ]:
dataset_file = "mycoco" # alternatively, implement your own coco-type dataset loader in datasets and add this "key" to datasets/__init__.py
#dataDir = dataset_train
dataDir='D:/NC_LAB/Trash_Classification/Code/DETR/splits_final_deblurred/' # should lead to a directory with a train2017 and val2017 folder as well as an annotations folder
num_classes = 5 # this int should be the actual number of classes + 1 (for no class)

outDir = 'D:/NC_LAB/Trash_Classification/Code/DETR/DETR_Output'
resume = "D:/NC_LAB/Trash_Classification/Code/DETR/detr-r50_training.pth"

In [ ]:
type(num_classes)

In [ ]:
!python -m D:/NC_Lab/Trash_Classification/Code/DETR/ main --dataset_file dataset_file --coco_path dataDir --output_dir outDir --resume resume --lr 1e-5 --lr_backbone 1e-6 --epochs 1